In [ ]:
# installez selenium si vous ne l'avais pas
!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Scrapping

In [9]:
# ouvrir le cite indeed en utilisant chromedriver
driver=webdriver.Chrome('chromedriver')
driver.get('https://fr.indeed.com/cmp/Soci%C3%A9t%C3%A9-G%C3%A9n%C3%A9rale-1/reviews')

In [ ]:
from selenium.webdriver.remote.errorhandler import NoSuchElementException
fichier = open("sg.txt", "w", encoding="utf-8")
i = 1

suivant_xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li/a'

while(True):
    xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/div[' + str(i) + ']'
    try :
        metier = driver.find_element(By.XPATH, xpath)
        fichier.write(metier.text+ 2*'\n')
        
        sep = 10*'#'
        fichier.write(sep + 2*'\n')
        i=i+1
        
    except NoSuchElementException : 
        time.sleep(60)
        suivant = driver.find_element(By.XPATH, suivant_xpath)
        suivant_xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li[2]/a'
        suivant.click()
        i=1


In [ ]:
fichier.close()

# Création de Base de donnée 

In [6]:
import pandas as pd

In [7]:
fichier = open('sg.txt', 'r', encoding="utf-8")
text = fichier.read()
parts = text.split('\n##########\n\n')

In [28]:
errors = {
    'rating': [],
    'job_city_date': []        
}

df = pd.DataFrame(columns= ['rating', 'contract_or_title', 'job', 'city', 'date', 'comment'])

for avis in parts:
    new_row = {}
    avis_par_ligne = avis.split('\n')
    i = 0
    
    rating = avis_par_ligne[i]
    # convertir par example '3,0' à 3.0
    rating = rating.replace(',', '.')
    try:
        # si la premiére ligne correspond à un classement on enregistre
        rating = float(rating)

        # passage à la ligne suivante
        i+=1

    except ValueError:
        errors['rating'].append(avis)
        continue

    contract_or_title = avis_par_ligne[i]
    i+=1

    job_city_date = avis_par_ligne[i]

    try :
        job, city, date = job_city_date.split(' - ')
        i+=1

    except ValueError : 

        errors['job_city_date'].append(avis)
        continue

    comment = ''
    while avis_par_ligne[i] in ['Avis à la une', "L'avis le plus utile, sélectionné par Indeed"]:
        i+=1
 


    while i<len(avis_par_ligne) and  avis_par_ligne[i]!= 'Avez-vous trouvé cet avis utile ?':
        
        comment += avis_par_ligne[i] + '\n'
        i+=1
        
    new_row = {
        'rating': rating,
        'contract_or_title': contract_or_title,
        'job': job,
        'city': city,
        'date': date,
        'comment': comment
    }
    df = df.append(new_row, ignore_index=True)

fichier.close()

In [29]:
df

,rating,contract_or_title,job,city,date,comment
0,3.0,Cdd,Conseillère clientèle (ancien employé),Massy (91),17 mai 2022,"Bonne équipe, dommage que la direction soit bl..."
1,5.0,Une équipe au top,Responsable (ancien employé),Falaise,28 novembre 2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,4.0,agréable,Compliance Officer (ancien employé),Nanterre (92),25 novembre 2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,5.0,Très bonne expérience,Assistante administrative (ancien employé),Saint-Dié-des-Vosges (88),24 novembre 2022,"Missions variées, solidarité dans l’équipe et ..."
4,4.0,Expérience au siège,Chargée de communication (ancien employé),La Défense (92),24 novembre 2022,Première expérience professionnelle durant 3 a...
...,...,...,...,...,...,...
1847,4.0,Très agréable et formateur,stagiaire analyste financier (ancien employé),Tour Société Générale à la défense,14 décembre 2012,Stage d'analyse financière dans le secteur imm...
1848,4.0,Cool,Financial Analyst (ancien employé),"Cergy, France",4 décembre 2012,I love my co workers.\nI learnt a lot about ba...
1849,4.0,Environnement de travail,Fiscaliste Internationale en Back Office (anci...,Paris,14 novembre 2012,excellent environnement de travail.\n\nUne exp...
1850,4.0,environnement excellent,conseiller de clientèle (ancien employé),grenoble,7 novembre 2012,"environnement agréable,bonnes équipes,travail ..."


In [31]:
len(errors['rating'])

453